In [1]:
import numpy as np
from PIDController import PIDController  # Import your PID controller
from OT2Eenv import OT2Env       # Import your environment
import matplotlib.pyplot as plt

env = OT2Env()
image_path = env.image()
print(f"Image saved at: {image_path}")


Image captured and saved at: textures/_plates/035_43-17-ROOT1-2023-08-08_mock_pH5_+Fe_Col0_04-Fish Eye Corrected.png
Image saved at: textures/_plates/035_43-17-ROOT1-2023-08-08_mock_pH5_+Fe_Col0_04-Fish Eye Corrected.png


In [2]:
def dice_loss(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    dice = (2. * intersection + K.epsilon()) / (union + K.epsilon())
    return 1 - dice
def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives+K.epsilon())
        return recall
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives+K.epsilon())
        return precision
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [3]:

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

from skimage.morphology import skeletonize
from PIDController import PIDController
from OT2Eenv import OT2Env  # Updated environment class


def crop_petri_dish(image, patch_size):
    """
    Detect and crop the Petri dish from the image.

    Parameters:
    - image: Input image (numpy array).
    - patch_size: Tuple (height, width) to pad the cropped Petri dish.

    Returns:
    - Cropped image focused on the Petri dish.
    - Bounding box of the Petri dish.
    - Success flag.
    """
    # Threshold the image to create a binary mask
    _, binary = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Detect the largest contour as the Petri dish
    largest_contour = max(contours, key=cv2.contourArea, default=None)
    if largest_contour is None:
        print("Error: No Petri dish detected.")
        return None, None, False

    # Get the bounding box of the Petri dish
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Crop the image based on the bounding box
    cropped_image = image[y:y + h, x:x + w]

    # Pad the cropped image to ensure it matches the patch size
    padded_image = pad_image(cropped_image, patch_size)

    return padded_image, (x, y, w, h), True


def pad_image(image, patch_size):
    """
    Pad the cropped image to match the required patch size.

    Parameters:
    - image: Input cropped image (numpy array).
    - patch_size: Tuple (height, width) for padding.

    Returns:
    - Padded image.
    """
    height, width = image.shape[:2]
    pad_height = (patch_size[0] - height % patch_size[0]) % patch_size[0]
    pad_width = (patch_size[1] - width % patch_size[1]) % patch_size[1]
    return cv2.copyMakeBorder(image, 0, pad_height, 0, pad_width, cv2.BORDER_CONSTANT, value=0)

def generate_mask(image, model, patch_size):
    """
    Generate a binary mask from an input image using the trained model.

    Parameters:
    - image: Input image (numpy array).
    - model: Trained Keras model.
    - patch_size: Tuple (height, width) for patching.

    Returns:
    - mask: Predicted binary mask (numpy array).
    """
    height, width = image.shape[:2]
    # Pad the image
    pad_height = (patch_size[0] - height % patch_size[0]) % patch_size[0]
    pad_width = (patch_size[1] - width % patch_size[1]) % patch_size[1]
    padded_image = cv2.copyMakeBorder(image, 0, pad_height, 0, pad_width, cv2.BORDER_CONSTANT, value=0)

    # Patch the image
    patches = []
    for y in range(0, padded_image.shape[0], patch_size[0]):
        for x in range(0, padded_image.shape[1], patch_size[1]):
            patch = padded_image[y:y + patch_size[0], x:x + patch_size[1]]
            patches.append(patch / 255.0)  # Normalize

    patches = np.array(patches)[..., np.newaxis]  # Add channel dimension

    # Predict patches
    predicted_patches = model.predict(patches)
    predicted_patches = (predicted_patches > 0.5).astype(np.uint8) * 255

    # Reconstruct the mask
    reconstructed_mask = np.zeros_like(padded_image, dtype=np.uint8)
    idx = 0
    for y in range(0, padded_image.shape[0], patch_size[0]):
        for x in range(0, padded_image.shape[1], patch_size[1]):
            reconstructed_mask[y:y + patch_size[0], x:x + patch_size[1]] = predicted_patches[idx].squeeze()
            idx += 1

    # Crop back to the original size
    return reconstructed_mask[:height, :width]


def detect_root_tip_from_skeleton(mask):
    """
    Detect the root tip (lowest point) in the skeletonized mask.

    Parameters:
    - mask: Binary mask (numpy array).

    Returns:
    - root_tip: Tuple (y_pixel, x_pixel) of the root tip.
    """
    # Skeletonize the mask
    skeleton = skeletonize(mask // 255)

    # Find the lowest non-zero pixel in the skeleton
    skeleton_pixels = np.argwhere(skeleton > 0)
    if skeleton_pixels.size == 0:
        raise ValueError("No root tip detected in skeleton.")

    root_tip = skeleton_pixels[np.argmax(skeleton_pixels[:, 0])]  # Lowest point (max y-coordinate)
    return tuple(root_tip)


def convert_pixel_to_mm(root_tip_pixel, image_height, plate_height_mm):
    """
    Convert pixel coordinates to millimeters.

    Parameters:
    - root_tip_pixel: Root tip coordinates in pixels (y_pixel, x_pixel).
    - image_height: Height of the original image in pixels.
    - plate_height_mm: Real-world height of the plate in millimeters.

    Returns:
    - root_tip_mm: Root tip coordinates in millimeters (x_mm, y_mm, z_mm).
    """
    scale = plate_height_mm / image_height  # mm per pixel
    y_mm = root_tip_pixel[0] * scale
    x_mm = root_tip_pixel[1] * scale
    return (x_mm, y_mm, 0)  # Assuming z=0 for simplicity


def convert_to_robot_coordinates(root_tip_mm, plate_position_robot):
    """
    Convert root tip positions in mm (relative to the plate) to robot space.

    Parameters:
    - root_tip_mm: Root tip position in mm (x_mm, y_mm, z_mm).
    - plate_position_robot: Position of the top-left corner of the plate in robot space [x, y, z].

    Returns:
    - root_tip_robot: Root tip position in robot space (x_robot, y_robot, z_robot).
    """
    return [
        root_tip_mm[0] + plate_position_robot[0],
        root_tip_mm[1] + plate_position_robot[1],
        root_tip_mm[2] + plate_position_robot[2],
    ]


def inoculate_with_pid(env, root_tips_robot):
    """
    Perform root tip inoculation using the PID controller.

    Parameters:
    - env: The simulation environment.
    - root_tips_robot: List of root tip coordinates in robot space [(x, y, z), ...].
    """
    # Initialize PID controllers for X, Y, Z axes
    pid_x = PIDController(kp=1.0, ki=0.1, kd=0.01)
    pid_y = PIDController(kp=1.0, ki=0.1, kd=0.01)
    pid_z = PIDController(kp=1.0, ki=0.1, kd=0.01)

    for root_tip in root_tips_robot:
        terminated = False
        truncated = False
        target_position = np.array(root_tip)

        while not (terminated or truncated):
            current_position = np.array(env.get_current_position())  # Robot's current position

            # Compute errors
            error_x = target_position[0] - current_position[0]
            error_y = target_position[1] - current_position[1]
            error_z = target_position[2] - current_position[2]

            # PID outputs
            control_x = pid_x.compute(error_x)
            control_y = pid_y.compute(error_y)
            control_z = pid_z.compute(error_z)

            # Take action
            action = np.array([control_x, control_y, control_z], dtype=np.float32)
            obs, reward, terminated, truncated, info = env.step(action)

            # Check if the robot is within the acceptable error threshold
            if np.linalg.norm(target_position - current_position) < 0.01:  # Example threshold
                print(f"Inoculating at {target_position}")
                env.drop_inoculum()  # Perform inoculation
                break

# Main Workflow
if __name__ == "__main__":
    # Parameters
    model_path =r"C:\Users\Edopi\Desktop\2024-25b-fai2-adsai-EdoardoPierezza231412\datalab_tasks\Task8\Edoardo_231412_undet_model256px_base.h5"
    patch_size = (256, 256)
    plate_position_robot = [0.10775, 0.088 - 0.026, 0.057]  # Adjusted plate position
    image_height = 2816  # Original image height in pixels
    plate_height_mm = 150  # Plate height in millimeters

    # Initialize the environment
    env = OT2Env()

    # Capture the image
    image_path = env.image()
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Crop the Petri dish
    cropped_image, bbox, success = crop_petri_dish(image, patch_size)
    if not success:
        raise RuntimeError("Failed to detect and crop the Petri dish.")

    # Load the trained model
    model = load_model(model_path, custom_objects={"f1": f1, "dice_loss": dice_loss})

    # Generate the mask
    mask = generate_mask(cropped_image, model, patch_size)

    # Detect the root tip
    root_tip_pixel = detect_root_tip_from_skeleton(mask)

    # Convert to mm
    root_tip_mm = convert_pixel_to_mm(root_tip_pixel, image_height, plate_height_mm)

    # Convert to robot coordinates
    root_tip_robot = convert_to_robot_coordinates(root_tip_mm, plate_position_robot)

    # Use the PID controller for inoculation
    inoculate_with_pid(env, [root_tip_robot])


Image captured and saved at: textures/_plates/030_43-2-ROOT1-2023-08-08_pvdCherry_OD001_Col0_05-Fish Eye Corrected.png
4/4 [==============================] - 8s 2s/step
Step 1 called.


AttributeError: 'OT2Env' object has no attribute 'goal_position'

In [ ]:


def generate_mask(image, model, patch_size):
    """
    Generate a mask from an image using the trained model.

    Parameters:
    - image: Input image (numpy array).
    - model: Trained Keras model.
    - patch_size: Tuple (height, width) for patching.

    Returns:
    - mask: Predicted binary mask (numpy array).
    """
    height, width = image.shape[:2]
    # Pad the image
    pad_height = (patch_size[0] - height % patch_size[0]) % patch_size[0]
    pad_width = (patch_size[1] - width % patch_size[1]) % patch_size[1]
    padded_image = cv2.copyMakeBorder(image, 0, pad_height, 0, pad_width, cv2.BORDER_CONSTANT, value=0)

    # Patch the image
    patches = []
    for y in range(0, padded_image.shape[0], patch_size[0]):
        for x in range(0, padded_image.shape[1], patch_size[1]):
            patch = padded_image[y:y + patch_size[0], x:x + patch_size[1]]
            patches.append(patch / 255.0)  # Normalize

    patches = np.array(patches)[..., np.newaxis]  # Add channel dimension

    # Predict patches
    predicted_patches = model.predict(patches)
    predicted_patches = (predicted_patches > 0.5).astype(np.uint8) * 255

    # Reconstruct the mask
    reconstructed_mask = np.zeros_like(padded_image, dtype=np.uint8)
    idx = 0
    for y in range(0, padded_image.shape[0], patch_size[0]):
        for x in range(0, padded_image.shape[1], patch_size[1]):
            reconstructed_mask[y:y + patch_size[0], x:x + patch_size[1]] = predicted_patches[idx].squeeze()
            idx += 1

    # Crop back to the original size
    return reconstructed_mask[:height, :width]

def detect_root_tip_from_skeleton(mask):
    """
    Detect the root tip (lowest point) in the skeletonized mask.

    Parameters:
    - mask: Binary mask (numpy array).

    Returns:
    - root_tip: Tuple (y_pixel, x_pixel) of the root tip.
    """
    # Skeletonize the mask
    skeleton = skeletonize(mask // 255)

    # Find the lowest non-zero pixel in the skeleton
    skeleton_pixels = np.argwhere(skeleton > 0)
    if skeleton_pixels.size == 0:
        raise ValueError("No root tip detected in skeleton.")

    root_tip = skeleton_pixels[np.argmax(skeleton_pixels[:, 0])]  # Lowest point (max y-coordinate)
    return tuple(root_tip)

def convert_pixel_to_mm(root_tip_pixel, image_height, plate_height_mm):
    """
    Convert pixel coordinates to millimeters.

    Parameters:
    - root_tip_pixel: Root tip coordinates in pixels (y_pixel, x_pixel).
    - image_height: Height of the original image in pixels.
    - plate_height_mm: Real-world height of the plate in millimeters.

    Returns:
    - root_tip_mm: Root tip coordinates in millimeters (x_mm, y_mm, z_mm).
    """
    scale = plate_height_mm / image_height  # mm per pixel
    y_mm = root_tip_pixel[0] * scale
    x_mm = root_tip_pixel[1] * scale
    return (x_mm, y_mm, 0)  # Assuming z=0 for simplicity

def convert_to_robot_coordinates(root_tip_mm, plate_position_robot):
    """
    Convert root tip positions in mm (relative to the plate) to robot space.

    Parameters:
    - root_tip_mm: Root tip position in mm (x_mm, y_mm, z_mm).
    - plate_position_robot: Position of the top-left corner of the plate in robot space [x, y, z].

    Returns:
    - root_tip_robot: Root tip position in robot space (x_robot, y_robot, z_robot).
    """
    return [
        root_tip_mm[0] + plate_position_robot[0],
        root_tip_mm[1] + plate_position_robot[1],
        root_tip_mm[2] + plate_position_robot[2],
    ]

def inoculate_with_pid(env, root_tips_robot):
    """
    Perform root tip inoculation using the PID controller.

    Parameters:
    - env: The simulation environment.
    - root_tips_robot: List of root tip coordinates in robot space [(x, y, z), ...].
    """
    # Initialize PID controllers for X, Y, Z axes
    pid_x = PIDController(kp=1.0, ki=0.1, kd=0.01)
    pid_y = PIDController(kp=1.0, ki=0.1, kd=0.01)
    pid_z = PIDController(kp=1.0, ki=0.1, kd=0.01)

    for root_tip in root_tips_robot:
        terminated = False
        truncated = False
        target_position = np.array(root_tip)

        while not (terminated or truncated):
            current_position = np.array(env.get_current_position())  # Robot's current position

            # Compute errors
            error_x = target_position[0] - current_position[0]
            error_y = target_position[1] - current_position[1]
            error_z = target_position[2] - current_position[2]

            # PID outputs
            control_x = pid_x.compute(error_x)
            control_y = pid_y.compute(error_y)
            control_z = pid_z.compute(error_z)

            # Take action
            action = np.array([control_x, control_y, control_z], dtype=np.float32)
            obs, reward, terminated, truncated, info = env.step(action)

            # Check if the robot is within the acceptable error threshold
            if np.linalg.norm(target_position - current_position) < 0.01:  # Example threshold
                print(f"Inoculating at {target_position}")
                env.drop_inoculum()  # Perform inoculation
                break

# Main Workflow
if __name__ == "__main__":
    # Parameters
    model_path = "path_to_your_trained_model.h5"
    patch_size = (256, 256)
    plate_position_robot = [0.10775, 0.088 - 0.026, 0.057]  # Adjusted plate position
    image_height = 2816  # Original image height in pixels
    plate_height_mm = 150  # Plate height in millimeters

    # Initialize the simulation environment
    env = RobotEnv()

    # Get the image from the simulation
    image = env.get_plate_image()

    # Load the trained model
    model = load_model(model_path)

    # Generate the mask
    mask = generate_mask(image, model, patch_size)

    # Detect the root tip
    root_tip_pixel = detect_root_tip_from_skeleton(mask)

    # Convert to mm
    root_tip_mm = convert_pixel_to_mm(root_tip_pixel, image_height, plate_height_mm)

    # Convert to robot coordinates
    root_tip_robot = convert_to_robot_coordinates(root_tip_mm, plate_position_robot)

    # Use the PID controller for inoculation
    inoculate_with_pid(env, [root_tip_robot])
